In [1]:
from datetime import datetime
from geopy.distance import great_circle
from operator import itemgetter
import matplotlib.pylab as plt
from pandas import DataFrame
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import datapane as dp
import pandas as pd
import random
import sys
import time
import numpy
import math

DELTA_GNSS = 0.05 #equivalent to about 16-17 km

#six decimal are enough for a decimeter precision
#0.1 = 11.1 km
#0.01 = 1.11 km
#...
#0.000001 = 0.111 m
uav_latitude = 45.0059289
uav_longitude = 10.4610255

#support variables
bts_latitude = 0
bts_longitude = 0

#Load BTS database
BTS_DATA = pd.read_csv('bts_windtre.csv', sep=';')
BTS_DATA = BTS_DATA.drop(columns=['tech','mcc','mnc','lac_tac','cid','psc_pci','arfcn','azimuth','height','tilt_mech','tilt_el'])

In [2]:
BTS_DATA = BTS_DATA.drop_duplicates(subset=['site_name'])

Nearby_Cells_Lat = abs(BTS_DATA['cell_lat']-uav_latitude)<=DELTA_GNSS
Nearby_Cells_Lon = abs(BTS_DATA['cell_long']-uav_longitude)<=DELTA_GNSS

NEARBY_BTS = BTS_DATA[Nearby_Cells_Lon]
NEARBY_BTS = NEARBY_BTS[Nearby_Cells_Lat]

NEARBY_BTS

/var/folders/jr/9x05_npj40g1pfxmcv1f95yc0000gn/T/ipykernel_77489/3830023252.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  NEARBY_BTS = NEARBY_BTS[Nearby_Cells_Lat]


,node_id,band,site_name,cell_lat,cell_long,cell_name
31518,116049,1800,Casalmaggiore Via Vanoni,44.982973,10.432590,B3 S1 DSS Casalmaggiore Via Vanoni
31764,116061,1800,Casalmaggiore,44.970562,10.464893,B3 S1 DSS Casalmaggiore
32146,116093,1800,Rivarolo del Re ed Uniti Via Belfiore,45.033475,10.484554,B3 S1 DSS Rivarolo del Re ed Uniti Via Belfiore
32764,116359,1800,Casalmaggiore Via Marconi,44.996525,10.421224,B3 S1 DSS Casalmaggiore Via Marconi
32788,116360,1800,Casalmaggiore Via Saffi,44.985169,10.414842,B3 S1 DSS Casalmaggiore Via Saffi
54806,124049,1800,Sabbioneta Via San Remigio,45.006472,10.491913,B3 S1 DSS Sabbioneta Via San Remigio
454166,524049,1800,Sabbioneta,45.006473,10.491917,B3 S1 DSS Sabbioneta


In [17]:
#Compute distance of nearby cells

uav_coo = (uav_latitude, uav_longitude)
nearby_bts_distances = []

for i in range(len(NEARBY_BTS)):
    cell = NEARBY_BTS.iloc[i]
    bts_coo = (cell.cell_lat, cell.cell_long)
    nearby_bts_distances.append(great_circle(uav_coo, bts_coo).m)

NEARBY_BTS['distance'] = nearby_bts_distances

NEARBY_BTS.sort_values(by='distance')

,node_id,band,site_name,cell_lat,cell_long,cell_name,distance
54806,124049,1800,Sabbioneta Via San Remigio,45.006472,10.491913,B3 S1 DSS Sabbioneta Via San Remigio,2429.097850
454166,524049,1800,Sabbioneta,45.006473,10.491917,B3 S1 DSS Sabbioneta,2429.375676
32764,116359,1800,Casalmaggiore Via Marconi,44.996525,10.421224,B3 S1 DSS Casalmaggiore Via Marconi,3299.473100
31518,116049,1800,Casalmaggiore Via Vanoni,44.982973,10.432590,B3 S1 DSS Casalmaggiore Via Vanoni,3393.436050
32146,116093,1800,Rivarolo del Re ed Uniti Via Belfiore,45.033475,10.484554,B3 S1 DSS Rivarolo del Re ed Uniti Via Belfiore,3577.992259
31764,116061,1800,Casalmaggiore,44.970562,10.464893,B3 S1 DSS Casalmaggiore,3944.393488
32788,116360,1800,Casalmaggiore Via Saffi,44.985169,10.414842,B3 S1 DSS Casalmaggiore Via Saffi,4303.105397
